[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neosatrapahereje/music_alignment_tutorial/blob/main/Symbolic_Music_Alignment.ipynb)



In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Install python packages
    ! pip install partitura
    ! pip install fastdtw

    # To be able to access helper modules in the repo for this tutorial
    # (not necessary if the jupyter notebook is run locally instead of google colab)
    !git clone https://github.com/cpjku/vienna4x22.git
    !git clone https://github.com/neosatrapahereje/music_alignment_tutorial
    import sys
    sys.path.insert(0, "./music_alignment_tutorial/")

# Symbolic Music Alignment

Automatic Music Alignment refers to the task of linking or matching two musical signals of the same musical work. This can be, e.g., matching *different performances* of the same piece, or matching the performance of a piece with its musical score.

<img src="figures/alignment_figure.gif" alt="alignment_figure" width="600"/>

The following figure shows a common music alignment pipeline:

<img src="figures/alignment_pipeline.png" alt="alignment_pipeline" width="600"/>

In this notebook we are going to explore these components in more detail.

In [ ]:
# Let's start by importing some stuff
import os 
# import glob
import warnings

import numpy as np
import matplotlib.pyplot as plt
import partitura as pt

from alignment import fast_dynamic_time_warping, greedy_note_alignment

from typing import List

warnings.filterwarnings("ignore")
%config InlineBackend.figure_format ='retina'

if IN_COLAB:
    V4X22_DATASET_DIR = "./vienna4x22"
else:
    # Path to the Vienna 4x22 dataset
    from load_data import init_dataset

    V4X22_DATASET_DIR = init_dataset()

MUSICXML_DIR = os.path.join(V4X22_DATASET_DIR, "musicxml")
MIDI_DIR = os.path.join(V4X22_DATASET_DIR, "midi")
MATCH_DIR = os.path.join(V4X22_DATASET_DIR, "match")



## 1. Music Representation

In this tutorial we will focus on symbolic music representations, such that can be stored in formats such as MIDI, MusicXML or MEI, and that can be generated by editors like MuseScore, Finale, etc.

### 1.1 Audio vs. Symbolic Alignment

* In **Audio-based alignment**, the alignment itself typically refers to  of *timestamps* (in absolute time in seconds) in one audio recording of a musical work to the corresponding *timestamp* in another recording. (In audio recordings, identifying individual notes is not a trivial task)


* In **Symbolic-based alignment**, we can have two types of alignment:
    * **Time-wise alignments**: similar to audio-based alignment, we can map timestamps (in symbolic time units like musical beats or MIDI ticks) from one version of the work to another (e.g., a MIDI performance to a score in MusicXML/MEI/Humdrum format). 
    * **Note-wise alignment**: We can map individual symbolic music elements (most commonly notes) from one version to another. This is very useful for modeling expressive performance.

### 1.2 Time

* **Offline**: Alignment of two *recordings/documents* (i.e., audio recordings, MIDI performances, MusicXML scores, etc.). These recordings/documents can be in any of the modalities described above, the important thing being that the music is occurring in real-time.

* **Online**: Alignment of a live (i.e., real time) performance to the music encoded in a target document (e.g., a pre-annotated audio recording, a symbolic score, etc.). The problem of real time online alignment is known in the MIR literature a **score following**, and can be useful in live interactive settings, such as automatic accompaniment systems

In this tutorial we are going to focus on the case of offline alignment.

## 2. Feature Representations

To make musical data comparable for alignment algorithms, the first step is to extract features that capture relevant aspects while suppressing irrelevant details.

In this lecture we are going to focus on piano rolls, one of the most commonly used features in symbolic music processing.

### Piano Rolls

A piano roll is a 2D representation of (MIDI) pitch and time. We can extract piano rolls from symbolic music files with Partitura!

In [ ]:
# Let's load a score and a performance of the score

# Path to the MusicXML file
score_fn = os.path.join(MUSICXML_DIR, "Chopin_op10_no3.musicxml")
performance_fn = os.path.join(MIDI_DIR, "Chopin_op10_no3_p01.mid")

score = pt.load_score(score_fn)
performance = pt.load_performance(performance_fn)

In [ ]:
# Compute piano roll
use_piano_range = False
score_pr = pt.utils.music.compute_pianoroll(
    note_info=score,
    piano_range=use_piano_range,
)

performance_pr = pt.utils.music.compute_pianoroll(
    note_info=performance,
    piano_range=use_piano_range,
)

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(2, figsize=(10, 7))
axes[0].imshow(
    score_pr.todense(),
    aspect="auto",
    origin="lower",
    cmap="gray",
    interpolation="nearest",
)
axes[1].imshow(
    performance_pr.todense(),
    aspect="auto",
    origin="lower",
    cmap="gray",
    interpolation="nearest",
)
y_label = "Piano key" if use_piano_range else "MIDI pitch"
axes[0].set_ylabel(y_label)
axes[1].set_ylabel(y_label)
axes[0].set_title("Score")
axes[1].set_title("Performance")
axes[1].set_xlabel("Time")
plt.show()

For more information, see the documentation of  [`compute_pianoroll`](https://partitura.readthedocs.io/en/latest/modules/partitura.utils.html#partitura.utils.compute_pianoroll).

## 3. Alignment Methods

We move now to methods for computing the alignment between features from one version of a piece of music to another. Common methods are dynamic programming approaches like dynamic time warping (DTW) and probabilistic approaches like hidden Markov models.

### 3.1 Dynamic Time Warping.

* DTW is a [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) algorithm to find the **optimal** alignment between to time-dependent sequences. 
* Unlike Euclidean distance, which requires point-to-point correspondence between two sequences, DTW allows for elastic transformations of the time axis, enabling it to find an optimal match between two sequences that may vary in time.
* The DTW algorithm finds the alignment between two sequence in three steps:

    1. Compute the pairwise distance between elements in sequence $\mathbf{X}$ (e.g., the performance) and $\mathbf{Y}$ (e.g., the score).
    2. Compute the *accumulated cost matrix* $\mathbf{D}$. The element $\mathbf{D}[i,j]$ represents the "cost" required for $x_i$ and $y_j$ to be aligned. For estimating this matrix, we use a recursive equation (this is the dynamic programming part!)
    3. Find the best alignment by backtracking, starting from the end of the performance.

Here is a more detailed tutorial on DTW: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neosatrapahereje/music_alignment_tutorial/blob/main/DTW_tutorial.ipynb)

## 4. Music Alignment with DTW

1. Compute features from score and the performance (in this case the piano rolls)
2. Compute the alignment between the sequences of features using DTW. This produces a time-wise alignment that tells us which time in the performance (in seconds) corresponds to which time in the score (in musical beats).
3. To get a note-level alignment, we match notes in the performance to notes in the score considering the time-wise alignment, and matching notes greedily by pitch.

In [ ]:
# This file contains the ground truth alignment
gt_alignment_fn = os.path.join(MATCH_DIR, "Chopin_op10_no3_p01.match")

# Load the alignment and the performance
performance, gt_alignment = pt.load_match(
    gt_alignment_fn, pedal_threshold=127, first_note_at_zero=True
)
pnote_array = performance.note_array()

# Load the score
score_fn = os.path.join(MUSICXML_DIR, "Chopin_op10_no3.musicxml")
score = pt.load_score(score_fn)
snote_array = score.note_array()


And now we compute the alignments using piano rolls.

In [ ]:
# Compute the features
score_pr, sidx = pt.utils.music.compute_pianoroll(
    note_info=score,
    time_unit="beat",
    time_div=8,
    return_idxs=True,
    piano_range=True,
    binary=True,
    note_separation=True,
)

performance_pr, pidx = pt.utils.music.compute_pianoroll(
    note_info=performance,
    time_unit="sec",
    time_div=8,
    return_idxs=True,
    piano_range=True,
    binary=True,
    note_separation=True,
)

reference_features = score_pr.toarray().T
performance_features = performance_pr.toarray().T

# DTW
dtw_pr_warping_path = fast_dynamic_time_warping(
    X=reference_features,
    Y=performance_features,
    metric="cityblock",
)

# Greedy note-level alignment
dtw_pr_alignment = greedy_note_alignment(
    warping_path=dtw_pr_warping_path,
    idx1=sidx,
    note_array1=snote_array,
    idx2=pidx,
    note_array2=pnote_array,
)


We can compare the performance of the alignment:

In [ ]:
from helper import evaluate_alignment_notewise

print(f"Method\tF-score\tPrecision\tRecall")

methods = [
    (dtw_pr_alignment, "DTW (piano roll)"),
]

for align, method in methods:
    precision, recall, fscore = evaluate_alignment_notewise(
        prediction=align,
        ground_truth=gt_alignment,
    )
    print(f"{method}\t{fscore:.4f}\t{precision:.4f}\t{recall:.4f}")

## 5. Alignment Applications: Comparing Expressive Performances

In this example, we are going to compare tempo curves of different performances of the same piece.

In [ ]:
from helper import plot_tempo_curves

# get all match files
piece = "Mozart_K331_1st-mov"
# piece = "Schubert_D783_no15"
# piece = "Chopin_op38"
# piece = "Chopin_op10_no3"

plot_tempo_curves(piece, V4X22_DATASET_DIR)

## 6. Summary

* In this notebook we learned about automatic music alignment for symbolic music
* We explored different parts of a pipeline of a system for alignment (features, alignment algorithms)
* We learned about the dynamic time warping algorithm, and how it can be used to align a performance to its score.
* Finally, we showed an application of alignment, focusing on comparing of expressive tempo curves

## 7. References for Further Reading

* Müller, M. (2021). [Music Synchronization](https://doi.org/10.1007/978-3-030-69808-9_3). In: Fundamentals of Music Processing. Springer, Cham. 
* Peter, S.D., Cancino-Chacón, C.E., Foscarin, F., McLeod, A.P., Henkel, F., Karystinaios, E. and Widmer, G., (2023). [Automatic Note-Level Score-to-Performance Alignments in the ASAP Dataset](https://transactions.ismir.net/articles/10.5334/tismir.149). Transactions of the International Society for Music Information Retrieval, 6(1), p.27–42.
* Foscarin, F., Karystinaios, E., Peter, S. D., Cancino-Chacón, C., Grachten, M., and Widmer, G.  (2022) [The match file format: Encoding Alignments between Scores and Performances](https://arxiv.org/pdf/2206.01104.pdf). In Proceedings of the Music Encoding Conference. Halifax, Canada
* Cancino-Chacón, C., Peter, S. D.,  Karystinaios, E., Foscarin, F., Grachten, M., and Widmer, G.  (2022) [Partitura: A Python Package for Symbolic Music Processing](https://arxiv.org/pdf/2206.01071.pdf). In Proceedings of the Music Encoding Conference. Halifax, Canada 
* Peter, S. D. (2023) [Online Symbolic Music Alignment With Offline Reinforcement Learning](https://archives.ismir.net/ismir2023/paper/000075.pdf). Proceedings of the 24th International Society for Music Information Retrieval Conference, 634–641. Milan, Italy